#### Andy Adamiec
#### Data Wrangleing Curse - Udacity
1/26/2017

---


#### Project Overview

The goal of this project was to use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for city of Chicago, IL. Once cleaned data is imported in to MongoDB database where further quries are issued to gain insight from this data. 

#### Project Objectives

* Assess the quality of the data for validity, accuracy, completeness, consistency and uniformity.

* Parse and gather data from popular file formats such as .csv, .json, .xml, and .html.

* Process data from multiple files or very large files that can be cleaned programmatically.

* Learn how to store, query, and aggregate data using MongoDB 

#### The Data Set

Data set used for this project is from OpenStreetMap for the city area of Chicago, IL and can be downloaded [here.](https://www.openstreetmap.org/relation/122604#map=10/41.8343/-87.7327)

Data primitives for this data are nodes, ways, and relations.

A **node** is one of the core elements in the OpenStreetMap data model. It consists of a single point in space defined by its latitude, longitude and node id.

A **way** is an ordered list of nodes which normally also has at least one tag or is included within a Relation. A way can have between 2 and 2,000 nodes, although it's possible that faulty ways with zero or a single node exist. A way can be open or closed. A closed way is one whose last node on the way is also the first on that way. A closed way may be interpreted either as a closed polyline, or an area, or both.

A **relation** is one of the core data elements that consists of one or more tags and also an ordered list of one or more nodes, ways and/or relations as members which is used to define logical or geographic relationships between other elements. A member of a relation can optionally have a role which describes the part that a particular feature plays within a relation.


##### Problems Encountered in the Map

* Abbreviated street types.

* Incorrect zip codes with missing digits and state abbreviation in zip code.

#### Tag Type 

To better understand the data I have used count_tag.py code to get the higher-level tag counts.

In [1]:
# Count number of tags in the file.

import xml.etree.cElementTree as ET
import pprint


def count_tags(filename):
    tags = {}
    
    for event, elem in ET.iterparse(filename,events=('start', 'end')):
        if elem.tag in tags and event == 'start':
            tags[elem.tag] += 1
        elif event == 'start':
            tags[elem.tag] = 1
        elif event == 'end':
            elem.clear()
            
    return tags

count_tags('chicago_illinois.osm')

{'bounds': 1,
 'member': 100887,
 'nd': 10539700,
 'node': 8900098,
 'osm': 1,
 'relation': 4869,
 'tag': 6877982,
 'way': 1265560}

I will be focusing on **node** and **way** tags of this data and associated **tag** tags.

#### Auditing the k tags.

We will first check "k" value for each "tag" and see if there are any potential problems. To do so we will first count the k tags in our data and look at the top 20 most frequent tags and then use three different regular expressions checking for following patterns: 

* "lower", for tags that contain only lowercase letters and are valid.
* "lower_colon", for otherwise valid tags with a colon in their names.
* "problemchars", for tags with problematic characters.
* "other", for other tags that do not fall into the other three categories.

#### Most frequent k tags in the data set.

Lets look at the top 20 k tags.

In [2]:
# function to sort dictionary
import operator

def sort_dict_val(d):
    sorted_x = sorted(d.items(), key=operator.itemgetter(1))
    sorted_x.reverse()
    return sorted_x

In [3]:
def audit(osmfile):
    tag_k = {}
    osm_file = open(osmfile, "r")
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "nodes" or elem.tag == "way":
            for tag in elem.iter("tag"):
                att = tag.attrib['k']
                if att in tag_k:
                    tag_k[att]+=1
                else:
                    tag_k[att] = 1

    return tag_k

In [4]:
sort_dict_val(audit('chicago_illinois.osm'))[:20]

[('building', 877653),
 ('chicago:building_id', 709050),
 ('addr:street', 487508),
 ('addr:housenumber', 484594),
 ('addr:street:name', 470573),
 ('addr:street:prefix', 466936),
 ('addr:street:type', 463807),
 ('building:levels', 421731),
 ('highway', 267502),
 ('name', 169861),
 ('tiger:county', 105621),
 ('tiger:cfcc', 105165),
 ('tiger:reviewed', 100355),
 ('tiger:name_base', 98296),
 ('tiger:name_type', 92647),
 ('tiger:zip_left', 81748),
 ('tiger:zip_right', 77542),
 ('service', 69774),
 ('source', 65610),
 ('oneway', 42687)]

Now we can count how many tags fall with in each category [lower,lower_colon,other,problemchars]

In [5]:
import re
from pprint import pprint

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        
        att = element.attrib['k'] 
        
        m = lower.search(att)
        o = problemchars.search(att)
        n = lower_colon.search(att)
        
        if m:
            keys['lower'] += 1
        elif o:
            keys['problemchars']+=1
        elif n:
            keys['lower_colon']+=1
        else:
            keys['other']+=1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

process_map('chicago_illinois.osm')

{'lower': 2147804, 'lower_colon': 3134339, 'other': 1595839, 'problemchars': 0}

In [6]:
def count_key_type(element, keys):
    if element.tag == 'tag':
        
        att = element.attrib['k']
        
        m = lower.search(att)
        o = problemchars.search(att)
        n = lower_colon.search(att)
        
        if m:
            if att in keys['lower']:
                keys['lower'][att] += 1
            else:
                keys['lower'][att] = 1
        elif o:
            if att in keys['problemchars']:
                keys['problemchars'][att] += 1
            else:
                keys['problemchars'][att] = 1
        elif n:
            if att in keys['lower_colon']:
                keys['lower_colon'][att] += 1
            else:
                keys['lower_colon'][att] = 1
        else:
            if att in keys['other']:
                keys['other'][att] += 1
            else:
                keys['other'][att] = 1
    return keys


def process_map(filename):
    keys = {"lower": {}, "lower_colon": {}, "problemchars": {}, "other": {}}
    for _, element in ET.iterparse(filename):
        keys = count_key_type(element, keys)
    return keys

data = process_map('chicago_illinois.osm')

Let take a look at the most frequent tags labeled as 'other'.

In [7]:
sort_dict_val(data['other'])[:20]

[('addr:street:name', 498511),
 ('addr:street:type', 492828),
 ('addr:street:prefix', 481555),
 ('tiger:name_base_1', 14654),
 ('name_1', 11617),
 ('tiger:name_type_1', 11311),
 ('NHD:FCode', 7637),
 ('NHD:ReachCode', 6627),
 ('NHD:ComID', 6584),
 ('tiger:name_direction_prefix_1', 5599),
 ('NHD:FTYPE', 4783),
 ('NHD:FDate', 4780),
 ('tiger:zip_left_1', 4582),
 ('turn:lanes:backward', 4429),
 ('turn:lanes:forward', 4411),
 ('NHD:RESOLUTION', 4274),
 ('tiger:name_base_2', 3171),
 ('NHD:way_id', 2852),
 ('NHD:FType', 2820),
 ('NHS', 2276)]

#### Explore Users.

Explore how many unique users contributed to the Chicago map.

In [8]:
def get_user(element):
    if element.tag == "node" or element.tag == "way" or element.tag == "relation":
        uid = element.get('uid')
        return uid


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if get_user(element):
            users.add(get_user(element))
    return users

len(process_map('chicago_illinois.osm'))

3098

#### Exploring street names.

We will audit street names to understand if there are any potential problems with the data. Lets first look at the sample of the street names in our data and then count the most frequent unexpected street types in the data set.

In [9]:
from collections import defaultdict

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def count_street_type(street_types_counts, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type =m.group()
        if street_type not in exprected:
            if street_type in street_types_counts:
                street_types_counts[street_type]+=1
            else:
                street_types_counts[street_type]=1 

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    street_types_counts={}
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

audit('chicago_illinois.osm')

defaultdict(set,
            {'1': {'Burlington Ave Ste 1',
              'Deerfield Pkwy #1',
              'Governors Hwy #1',
              'Lunt Avenue #1',
              'West North Avenue #1'},
             '103': {'Renaissance Drive #103'},
             '104': {'North Sherman Avenue #104'},
             '110': {'Commerce Drive #110'},
             '127': {'Skokie Boulevard #127'},
             '14': {'Route 14', 'U.S. 14', 'Us Highway 14'},
             '1425': {'N Lake Shore Dr #1425'},
             '176': {'IL Route 176', 'Route 176'},
             '180': {'Commerce Drive Suite 180'},
             '1850': {'E Madison St #1850'},
             '1B': {'W Newport Ave #1B'},
             '1E': {'West Waveland Avenue Apt 1E'},
             '20': {'US 20', 'West US Highway 20'},
             '200': {'Lake Street #200'},
             '201': {'Route 30, Suite 201'},
             '2105': {'North Michigan Avenue # 2105'},
             '22': {'E Route 22', 'IL 22', 'Il 22'},
             

Most frequent unexpected street types in the data set.

In [10]:
from collections import defaultdict

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]



def count_street_type(street_types_counts, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type =m.group()
        if street_type not in expected:
            if street_type in street_types_counts:
                street_types_counts[street_type]+=1
            else:
                street_types_counts[street_type]=1 

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    street_types_counts={}
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    count_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

sort_dict_val(audit('chicago_illinois.osm'))

[('L', 716),
 ('M', 712),
 ('N', 627),
 ('Dr', 627),
 ('Terrace', 559),
 ('J', 554),
 ('O', 509),
 ('H', 490),
 ('Highway', 444),
 ('G', 414),
 ('Broadway', 403),
 ('St', 299),
 ('F', 273),
 ('Ave', 262),
 ('Circle', 248),
 ('Ct', 241),
 ('E', 234),
 ('B', 192),
 ('C', 190),
 ('Park', 153),
 ('D', 140),
 ('Rd', 139),
 ('Way', 130),
 ('West', 116),
 ('Cir', 91),
 ('Ln', 90),
 ('K', 61),
 ('Market', 51),
 ('Center', 42),
 ('Blvd', 39),
 ('Plaza', 33),
 ('59', 21),
 ('North', 18),
 ('47', 17),
 ('Path', 15),
 ('Dr.', 15),
 ('Trl', 15),
 ('Pkwy', 13),
 ('St.', 10),
 ('Rd.', 10),
 ('Walk', 9),
 ('Ave.', 8),
 ('45', 7),
 ('Hwy', 5),
 ('1', 5),
 ('Ct.', 5),
 ('6', 5),
 ('64', 5),
 ('34', 5),
 ('83', 5),
 ('Crossing', 4),
 ('Steet', 4),
 ('Row', 4),
 ('30', 4),
 ('20', 4),
 ('Pl', 4),
 ('Shabbona', 3),
 ('South', 3),
 ('A', 3),
 ('East', 3),
 ('14', 3),
 ('Blvd.', 3),
 ('22', 3),
 ('st', 2),
 ('ave', 2),
 ('roosevelt', 2),
 ('500', 2),
 ('176', 2),
 ('38', 2),
 ('Mall', 2),
 ('Grove', 2),
 ('r

We notice few extra types which do not have any issues but are not in our expected list like: [Terrace,Broadway,Highway,Circle,Park,Plaza,Way,West].These will be added to the expected list. 

Some of the issues are as follows: [Ave,Pkwy,Dr,St,Ct,Rd.,Cir]. These will be added to the mapping file for correction.

#### Imroving Street Names

Now we can fix the error we found in the street names.

In [11]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "chicago_illinois.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons",'Terrace','Broadway','Highway','Circle','Park','Plaza','Way','West']

mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.":"Road",
            "Cir": "Circle",
            "Pkwy":"Parkway",
            "Dr":"Drive",
            "Ct": "Court"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected and street_type in mapping:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    m = street_type_re.search(name)
    street_type = m.group()
    mapping[street_type]
    return name.replace(street_type, mapping[street_type])


def test():
    st_types = audit(OSMFILE)

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name


test()

S. Arlington Heights Rd. => S. Arlington Heights Road
W. State Rd. => W. State Road
N. Rand Rd. => N. Rand Road
S Elmhurst Rd. => S Elmhurst Road
St. Charles Rd. => St. Charles Road
E. Rand Rd. => E. Rand Road
E. Boughton Rd. => E. Boughton Road
N. Wolf Rd. => N. Wolf Road
Oakton St. => Oakton Street
W. Lake St. => W. Lake Street
W. Madison St. => W. Madison Street
Valley View St. => Valley View Street
E 75th St. => E 75th Street
W 129th St. => W 129th Street
N. Main St. => N. Main Street
N. Clark St. => N. Clark Street
North Bridge St. => North Bridge Street
W Eames St. => W Eames Street
Mississippi Pkwy => Mississippi Parkway
W Countryside Pkwy => W Countryside Parkway
W Veterans Pkwy => W Veterans Parkway
Wesmere Pkwy => Wesmere Parkway
Veterans Pkwy => Veterans Parkway
E Countryside Pkwy => E Countryside Parkway
Wedgewood Cir => Wedgewood Circle
Aspen Cir => Aspen Circle
Glenbrook Cir => Glenbrook Circle
N Lincoln Ave => N Lincoln Avenue
N Cumberland Ave => N Cumberland Avenue
Inde

The output data looks good, the errors within the street names where fixed successfully.

Now lets audit postal codes in the data to see if there are any errors.

In [12]:
zipcode_re = re.compile(r'^\d{5}(?:[-\s]?\d{4})?$', re.IGNORECASE)


def audit_zip_code(zip_code_types, zip_code):
    m = zipcode_re.search(zip_code)
    if not m:
        #zip_code_type = m.group()
        zip_code_types[zip_code].add(zip_code)


def is_zip_code(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    zip_code_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zip_code(tag):
                    audit_zip_code(zip_code_types, tag.attrib['v'])
    osm_file.close()
    return zip_code_types

In [13]:
audit_zip("chicago_illinois.osm")

defaultdict(set,
            {'0201': {'0201'},
             '2300': {'2300'},
             '3333': {'3333'},
             '6007': {'6007'},
             '601412': {'601412'},
             '6017': {'6017'},
             '60183 / 60174': {'60183 / 60174'},
             '6051': {'6051'},
             '606': {'606'},
             '60614 ': {'60614 '},
             '606226': {'606226'},
             '6074': {'6074'},
             '690639': {'690639'},
             'IL': {'IL'},
             'IL 60118': {'IL 60118'},
             'IL 60605-1226': {'IL 60605-1226'},
             'IL 60707': {'IL 60707'},
             'IL, 60126': {'IL, 60126'},
             'IL, 60642': {'IL, 60642'},
             'Il': {'Il'},
             'None': {'None'},
             'River Forest': {'River Forest'},
             'Schaumburg': {'Schaumburg'},
             'Wasco, IL  60183': {'Wasco, IL  60183'},
             'Wasco, IL 60183': {'Wasco, IL 60183'}})

For the most part zip code are clean in our data. The above error are few compare to the size of data and I will replace them with a NULL value.

We are ready to save the data in JSON format and read it in to MongoDB data base.

In [14]:
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
zipcode_re = re.compile(r'^\d{5}(?:[-\s]?\d{4})?$', re.IGNORECASE)

mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.":"Road",
            "Cir": "Circle",
            "Pkwy":"Parkway",
            "Dr":"Drive",
            "Ct": "Court"}

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            return name.replace(street_type, mapping[street_type])
        else:
            return name
    
def update_zip(zip_code):
    m = zipcode_re.search(zip_code)
    if m:
        return zip_code
    else:
        return ''

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]



def shape_element(element):
    node = {}
    
    if element.tag == "node" or element.tag == "way":
        
        address_dic = {}
        nd_list = []
        
        for tag in element.iter("tag"):
            
            att = tag.attrib['k'] 
        
            if problemchars.search(att):
                continue
                
            elif tag.attrib['k'].startswith("addr:") and len(tag.attrib['k'].split(':')) < 3:
                
                if att.find('street')>-1:
                    address_dic['street']=update_name(tag.attrib['v'],mapping)
                    
                elif att.find('housenumber')>-1:
                    address_dic['housenumber']=tag.attrib['v']
                    
                elif att.find('postcode')>-1:
                    address_dic['postcode']=update_zip(tag.attrib['v'])
                    
            elif not tag.attrib['k'].startswith("addr:") and len(tag.attrib['k'].split(':')) < 3:
                temp_list = tag.attrib['k'].split(':')
                node[temp_list[-1]] = tag.attrib['v']
 
        if bool(address_dic):        
            node['address']  = address_dic    

        node['type'] = element.tag
        
        for tag in element.iter('nd'):
            nd_list.append(tag.attrib['ref'])
        
        if len(nd_list) > 0:
            node['node_refs'] = nd_list 
            
        created_dic = {}            
        pos_list = [None,None]
        
        for key,val in element.items():
            
            if key == 'lat':
                pos_list[0] = float(val)
            elif key == 'lon':
                pos_list[1] = float(val)
            elif key in CREATED:
                created_dic[key] = val
            else:
                node[key] = val
            
        node['created'] = created_dic
        node['pos'] = pos_list
        
        return node
    else:
        return None    


def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data



In [15]:
process_map('chicago_illinois.osm')

[{'created': {'changeset': '7781188',
   'timestamp': '2011-04-06T05:17:15Z',
   'uid': '207745',
   'user': 'NE2',
   'version': '54'},
  'exit_to': 'Joliet Road',
  'highway': 'motorway_junction',
  'id': '219850',
  'pos': [41.7585879, -87.9101245],
  'ref': '276C',
  'type': 'node'},
 {'created': {'changeset': '7781188',
   'timestamp': '2011-04-06T05:18:47Z',
   'uid': '207745',
   'user': 'NE2',
   'version': '47'},
  'exit_to': 'North I-294 ; Tri-State Tollway;  Wisconsin',
  'highway': 'motorway_junction',
  'id': '219851',
  'pos': [41.7593116, -87.9076432],
  'ref': '277A',
  'type': 'node'},
 {'created': {'changeset': '485405',
   'timestamp': '2009-04-13T11:21:51Z',
   'uid': '18480',
   'user': 'nickvet419',
   'version': '14'},
  'id': '700724',
  'pos': [41.7120272, -88.0158606],
  'type': 'node'},
 {'created': {'changeset': '485405',
   'timestamp': '2009-04-13T11:21:51Z',
   'uid': '18480',
   'user': 'nickvet419',
   'version': '14'},
  'id': '700725',
  'pos': [41.71

#### Load data in to MongoDB data-base.

In [16]:
import os
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.osm
collection = db.chicago

collection.drop()
os.system('mongoimport -d osm -c chicago chicago_illinois.osm.json')

0

File size.

In [17]:
print 'Chicago OSM file is {} MB'.format(os.path.getsize('chicago_illinois.osm')/1.0e6) 

Chicago OSM file is 2123.531129 MB


Number of documents.

In [18]:
db.chicago.find().count()

10165658

Number of unique users

In [19]:
len(db.chicago.distinct('created.user'))

3069

Number of nodes

In [20]:
db.chicago.find({"type":"node"}).count()

8900098

Number of ways

In [21]:
db.chicago.find({"type":"way"}).count()

1265560

Most popular cuisine.

In [22]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity":"restaurant", "cuisine":{"$exists":1}}}, 
            {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},        
            {"$sort":{"count":-1}}, 
            {"$limit":20}]
result = db.chicago.aggregate(pipeline)

for pp in list(result):
    print(pp)

{u'count': 198, u'_id': u'pizza'}
{u'count': 180, u'_id': u'mexican'}
{u'count': 136, u'_id': u'american'}
{u'count': 91, u'_id': u'italian'}
{u'count': 81, u'_id': u'burger'}
{u'count': 75, u'_id': u'chinese'}
{u'count': 54, u'_id': u'sandwich'}
{u'count': 43, u'_id': u'thai'}
{u'count': 34, u'_id': u'japanese'}
{u'count': 28, u'_id': u'indian'}
{u'count': 25, u'_id': u'sushi'}
{u'count': 24, u'_id': u'steak_house'}
{u'count': 23, u'_id': u'breakfast'}
{u'count': 22, u'_id': u'barbecue'}
{u'count': 21, u'_id': u'chicken'}
{u'count': 16, u'_id': u'greek'}
{u'count': 14, u'_id': u'asian'}
{u'count': 13, u'_id': u'ice_cream'}
{u'count': 13, u'_id': u'regional'}
{u'count': 11, u'_id': u'french'}


Most frequant zip code.

In [23]:
zip_code = db.chicago.aggregate( [ 
    { "$match" : { "address.postcode" : { "$exists" : 1} } }, 
    { "$group" : { "_id" : "$address.postcode", "count" : { "$sum" : 1} } },  
    { "$sort" : { "count" : -1}},
      {"$limit":20}] )


for pp in list(zip_code):
    print(pp)

{u'count': 9395, u'_id': u'60201'}
{u'count': 7620, u'_id': u'60202'}
{u'count': 1724, u'_id': u'60305'}
{u'count': 1685, u'_id': u'60564'}
{u'count': 1306, u'_id': u'60136'}
{u'count': 1259, u'_id': u'60042'}
{u'count': 909, u'_id': u'60637'}
{u'count': 868, u'_id': u'60089'}
{u'count': 657, u'_id': u'60148'}
{u'count': 536, u'_id': u'60565'}
{u'count': 460, u'_id': u'60521'}
{u'count': 396, u'_id': u'60615'}
{u'count': 370, u'_id': u'60005'}
{u'count': 303, u'_id': u'60004'}
{u'count': 296, u'_id': u'60008'}
{u'count': 276, u'_id': u'60611'}
{u'count': 267, u'_id': u'60067'}
{u'count': 266, u'_id': u'60173'}
{u'count': 266, u'_id': u'60056'}
{u'count': 265, u'_id': u'60076'}
